In [114]:
from rllab.algos.trpo import TRPO
from rllab.algos.batch_polopt import BatchSampler
from rllab.baselines.linear_feature_baseline import LinearFeatureBaseline
from rllab.envs.gym_env import GymEnv
from rllab.envs.normalized_env import normalize
from rllab.misc.instrument import run_experiment_lite
from rllab.misc.instrument import VariantGenerator, variant
from rllab.policies.categorical_mlp_policy import CategoricalMLPPolicy
from rllab.policies.gaussian_mlp_policy import GaussianMLPPolicy
from rllab.policies.deterministic_mlp_policy import DeterministicMLPPolicy
from rllab.sampler.utils import collect_data, rollout
from rllab.sampler import parallel_sampler


import gym
import numpy as np
import GPy
import GPyOpt

import matplotlib.pyplot as plt
%matplotlib inline

import matplotlib
import pandas as pd


import plotly
plotly.offline.init_notebook_mode() 
import plotly.plotly as py
import plotly.graph_objs as go

import numpy as np
import pandas as pd
import copy
import pickle
from datetime import datetime

import random
from operator import add, sub

In [25]:
env_sim.wrapped_env.env.env.model.geom_friction

array([[  1.00000000e+00,   5.00000000e-03,   1.00000000e-04],
       [  1.00000000e+00,   5.00000000e-03,   1.00000000e-04],
       [  1.00000000e+00,   5.00000000e-03,   1.00000000e-04],
       [  1.00000000e+00,   5.00000000e-03,   1.00000000e-04]])

In [119]:
def myf(X):
    X = X.T
    
#     mb = np.zeros((3,1))
#     mb[0,0] = float(0)
#     mb[1,0] = float(9.42477796)*0.6
#     mb[2,0] = float(4.8953899)*0.6
#     env_sim.wrapped_env.env.env.model.body_mass = mb

#     env_sim.wrapped_env.env.env.model.body_mass = env_real.wrapped_env.env.env.model.body_mass
    env_sim_BO.wrapped_env.env.env.model.body_mass = X
        
    dim = env_sim_BO.wrapped_env.env.env.model.data.qpos.shape[0]
#     print("states: ", states)
#     print("actions: ", actions)
#     print("observations: ", observations)    
    env_sim_BO.wrapped_env.env.env.set_state(states[0,:dim],states[0,dim:])
#     env_sim.wrapped_env.env.env.set_state(states[0,:2],states[0,2:])

    sim_states = []
    rewards = []
    env_sim_BO.reset()
    
    for i in range(0,actions.shape[0]):
        next_o, r, d, env_info = env_sim_BO.step(actions[i,:])
        rewards.append(r)        
        sim_states.append(next_o)
#     print(states)
#     print("sim_states: ", sim_states)
    dist = sum(sum(abs(observations[1:,:]-np.array(sim_states))))/observations.shape[0]
#     print("dist: ", dist)
#     print("rewards: ", sum(rewards))
    
    return dist

In [121]:
def BO(bounds, X_init, Y_init):
    print("bounds: ", bounds)
    print("X_init size: ", X_init.size)
    
    global env_sim_BO
    env_sim_BO = normalize(GymEnv(env_name,record_video=False, record_log=False))
    
    if X_init.size == 0:
        myBopt = GPyOpt.methods.BayesianOptimization(myf, domain=bounds)
    else:
        myBopt = GPyOpt.methods.BayesianOptimization(myf, domain=bounds, X = X_init, Y = Y_init)
        
    time_budget = 30
    print("running BO with time budget: ", time_budget)
    myBopt.run_optimization(max_time=time_budget)
    print("myBopt.x_opt: ", myBopt.x_opt, "\n\n\n")
    return myBopt

In [126]:
class VG(VariantGenerator):

    @variant
    def seed(self):
        return list(range(1, 41))

In [127]:
############# sim = [0.6,0.8]-[1.2,1.4]* real


global env_name
env_name = "Swimmer-v1"
returns = []
total_iter = 400
test_iter = 10
num_iters = int(total_iter/test_iter)

def run_task(*_):
    # Please note that different environments with different action spaces may
    # require different policies. For example with a Discrete action space, a
    # CategoricalMLPPolicy works, but for a Box action space may need to use
    # a GaussianMLPPolicy (see the trpo_gym_pendulum.py example)
    env_real = normalize(GymEnv(env_name,record_video=False, record_log=False))
    model_body_mass = env_real.wrapped_env.env.env.model.body_mass    
    print("real mass: ", model_body_mass)    
    bounds = []
    for i in range(0,model_body_mass.shape[0]):
        bounds.append({'name': 'mass', 'type': 'continuous', 'domain': (model_body_mass[i,0]*0.5,model_body_mass[i,0]*1.5)})
    
    env_sim = normalize(GymEnv(env_name,record_video=False, record_log=False))
    
    op = random.choice([add, sub])
    factor = op(1,0.5*np.random.random_sample())
    mb = env_sim.wrapped_env.env.env.model.body_mass
    mb = np.array(mb)
    mb = mb*factor
    env_sim.wrapped_env.env.env.model.body_mass = mb
    policy = GaussianMLPPolicy(
        env_spec=env_sim.spec,
        # The neural network policy should have two hidden layers, each with 32 hidden units.
        hidden_sizes=(32, 32)
    )
    
    X_init = np.empty(shape=(0, 0))
    Y_init = np.empty(shape=(0, 0))
    
    baseline = LinearFeatureBaseline(env_spec=env_sim.spec)
    
    algo = TRPO(
            env=env_sim,
            policy=policy,
            baseline=baseline,
            batch_size=4000,
            max_path_length=1000,
            n_itr=10,
            discount=0.99,
            step_size=0.01,
    #         Uncomment both lines (this and the plot parameter below) to enable plotting
    #         plot=True,
        )
    for i in range(0,num_iters):
        print("iter: ", i)
#         algo.n_itr = algo.n_itr + test_iter
        current_mass = algo.env.wrapped_env.env.env.model.body_mass
        print("algo env mass: ", current_mass)
        algo.train()
        
        ################### Testing in GT parameter #################
#         print("algo sim mass: ", algo.env.wrapped_env.env.env.model.body_mass)
#         sampler_cls = BatchSampler
#         sampler = sampler_cls(algo)
#         sampler.start_worker()
#         paths = sampler.obtain_samples(1)
#         paths_returns = [sum(path["rewards"]) for path in paths]
#         sampler.shutdown_worker()
#         print("Sim sample ave returns:  \n\n\n\n", np.mean(paths_returns))   
        
        
#         print("algo sim mass: ", algo.env.wrapped_env.env.env.model.body_mass)
#         parallel_sampler.populate_task(algo.env, algo.policy, scope=algo.scope)
#         paths = parallel_sampler.sample_paths(
#             policy_params=algo.policy.get_param_values(),
#             max_samples=algo.batch_size,
#             max_path_length=algo.max_path_length,
#             env_params=algo.env.get_param_values(),
#             scope=algo.scope,
#         )
#         paths_returns = [sum(path["rewards"]) for path in paths]
#         parallel_sampler.terminate_task(scope=algo.scope)
#         print("Sim sample ave returns: ", np.mean(paths_returns), "\n\n\n\n")   
                
        env_real = normalize(GymEnv(env_name,record_video=False, record_log=False))
        print("real mass: ", env_real.wrapped_env.env.env.model.body_mass)
        parallel_sampler.populate_task(env_real, algo.policy, scope=algo.scope)
        paths_real = parallel_sampler.sample_paths(
            policy_params=algo.policy.get_param_values(),
            max_samples=algo.batch_size,
            max_path_length=algo.max_path_length,
            env_params=env_real.get_param_values(),
            scope=algo.scope,
        )
        paths_real_returns = [sum(path["rewards"]) for path in paths_real]
        parallel_sampler.terminate_task(scope=algo.scope)
        ave_returns = np.mean(paths_real_returns)
        print("Real sample ave returns: ", ave_returns, "\n\n\n\n")
        returns.append(ave_returns)
        
#         algo_real = copy.deepcopy(algo)
#         algo_real.env = env_real
#         print("algo real mass: ", algo_real.env.wrapped_env.env.env.model.body_mass)
#         sampler_cls = BatchSampler
#         sampler = sampler_cls(algo_real)
#         sampler.start_worker()
#         paths = sampler.obtain_samples(1)
#         paths_returns = [sum(path["rewards"]) for path in paths]
#         sampler.shutdown_worker()
#         print("Real sample ave returns:", np.mean(paths_returns), "\n\n\n\n")   
#         returns.append(np.mean(paths_returns))
        
        env_real = normalize(GymEnv(env_name,record_video=False, record_log=False))
        print("real mass: ", env_real.wrapped_env.env.env.model.body_mass)
        path = collect_data(env_real, algo.policy, max_path_length=1000,
                       animated=False, speedup=1)
        path["returns"] = sum(path["rewards"])
        print("real returns:  ", path['returns'], path['rewards'].shape , "\n\n\n\n")
#         returns.append(path['returns'])
        
        global states, observations, actions
        states = path['states']
        observations = path['observations']
        actions = path['actions']

        print("sim mass: ", algo.env.wrapped_env.env.env.model.body_mass)
        path_2 = rollout(algo.env, algo.policy, max_path_length=1000,
                       animated=False, speedup=1)
        path_2["returns"] = sum(path_2["rewards"])
        print("sim returns: ", path_2['returns'], path_2["rewards"].shape, "\n\n\n\n") 
        
#         env_sim_test = normalize(GymEnv(env_name,record_video=False, record_log=False))
# #         env_sim_test.wrapped_env.env.env.model.body_mass = np.array(current_mass)
#         print("sim test mass: ", env_sim_test.wrapped_env.env.env.model.body_mass)
#         path_2 = rollout(env_sim_test, algo.policy, max_path_length=1000,
#                        animated=False, speedup=1)
#         path_2["returns"] = sum(path_2["rewards"])
#         print("sim test returns: ", path_2['returns'], path_2["rewards"].shape, "\n\n\n\n") 
        
#         if i==1:
#             model_body_mass = env_sim.wrapped_env.env.env.model.body_mass
#             bounds = []
#             for i in range(0,model_body_mass.shape[0]):
#                 bounds.append({'name': 'mass', 'type': 'continuous', 'domain': (model_body_mass[i,0]*0.9,model_body_mass[i,0]*1.1)})
        
        myBopt = BO(bounds, X_init, Y_init)
        X_init = myBopt.X
        Y_init = myBopt.Y
# #         algo.env.wrapped_env.env.env.model.body_mass = np.array(myBopt.x_opt)
        
        env_sim_new = normalize(GymEnv(env_name,record_video=False, record_log=False))        
#         ## update env_sim with new boday_mass from BO
        env_sim_new.wrapped_env.env.env.model.body_mass = np.array(myBopt.x_opt)
        algo = TRPO(
            env=env_sim_new,
            policy=policy,
            baseline=baseline,
            batch_size=4000,
            max_path_length=1000,
            n_itr=10,
            discount=0.99,
            step_size=0.01,
    #         Uncomment both lines (this and the plot parameter below) to enable plotting
    #         plot=True,
        )
        algo.env = env_sim_new        
        ##############################################################
        
    print("returns: ", returns)
    now = datetime.now()
    save_data_filename = "BO_returns_"+env_name+"_iter_"+str(total_iter)+"_"+str(now)
    with open(save_data_filename, 'wb') as f:
        pickle.dump(returns, f)


In [128]:
variants = VG().variants()
for v in variants:
    print("seed: ", v["seed"])
    run_experiment_lite(
        run_task,
        # Number of parallel workers for sampling
        n_parallel=1,
        # Only keep the snapshot parameters for the last iteration
        snapshot_mode="last",
        # Specifies the seed for the experiment. If this is not provided, a random seed
        # will be used
        seed=v["seed"],
        exp_prefix = "BO_trpo"+env_name+"_32_32_4000_10",
    #     plot=True,
    )

seed:  1
python /home/zhusj/repos/rllab/scripts/run_experiment_lite.py  --args_data 'gAJjY2xvdWRwaWNrbGUuY2xvdWRwaWNrbGUKX2ZpbGxfZnVuY3Rpb24KcQAoY2Nsb3VkcGlja2xlLmNsb3VkcGlja2xlCl9tYWtlX3NrZWxfZnVuYwpxAWNjbG91ZHBpY2tsZS5jbG91ZHBpY2tsZQpfYnVpbHRpbl90eXBlCnECWAgAAABDb2RlVHlwZXEDhXEEUnEFKEsASwBLGUsSS0djX2NvZGVjcwplbmNvZGUKcQZYcQUAAHQAAHQBAHQCAGQBAGQCAGQDAGQCAMKDAQLCgwEAfQEAfAEAagMAagQAagQAagUAagYAfQIAdAcAZAQAfAIAwoMCAAFnAAB9AwB4XwB0CABkBQB8AgBqCQBkBQAZwoMCAERdRwB9BAB8AwBqCgBkBgBkBwBkCABkCQBkCgB8AgB8BABkBQBmAgAZZAsAFHwCAHwEAGQFAGYCABlkDAAUZgIAaQMAwoMBAAFxXQBXdAAAdAEAdAIAZAEAZAIAZAMAZAIAwoMBAsKDAQB9BQB0CwBqDAB0DQB0DgBnAgDCgwEAfQYAfAYAZA0AZAsAdA8AagsAahAAwoMAABTCgwIAfQcAfAUAagMAagQAagQAagUAagYAfQgAdA8AahEAfAgAwoMBAH0IAHwIAHwHABR9CAB8CAB8BQBqAwBqBABqBABqBQBfBgB0EgBkDgB8BQBqEwBkDwBkOADCgwACfQkAdA8AahQAZBEAZDkAwoMAAX0KAHQPAGoUAGQRAGQ6AMKDAAF9CwB0FQBkDgB8BQBqEwDCgwABfQwAdBYAZBIAfAUAZBMAfAkAZBQAfAwAZBUAZBYAZBcAZBgAZBkAZBoAZBsAZBwAZB0AZB4AwoMACH0NAHgIA3QIAGQFAHQXAMKDAgBEXcO3An0EAHQHAGQfAHwEAMKDAgA

KeyboardInterrupt: 

In [59]:
############### real = 0.6 * sim

global env_name
env_name = "Swimmer-v1"
returns = []
total_iter = 400
test_iter = 10
num_iters = int(total_iter/test_iter)

def run_task(*_):
    # Please note that different environments with different action spaces may
    # require different policies. For example with a Discrete action space, a
    # CategoricalMLPPolicy works, but for a Box action space may need to use
    # a GaussianMLPPolicy (see the trpo_gym_pendulum.py example)
    env_sim = normalize(GymEnv(env_name,record_video=False, record_log=False))
    print("sim mass: ", env_sim.wrapped_env.env.env.model.body_mass)
    

    
    model_body_mass = env_sim.wrapped_env.env.env.model.body_mass
    bounds = []
    for i in range(0,model_body_mass.shape[0]):
        bounds.append({'name': 'mass', 'type': 'continuous', 'domain': (model_body_mass[i,0]*0.5,model_body_mass[i,0]*1.5)})
    

    policy = GaussianMLPPolicy(
        env_spec=env_sim.spec,
        # The neural network policy should have two hidden layers, each with 32 hidden units.
        hidden_sizes=(32, 32)
    )
    
    X_init = np.empty(shape=(0, 0))
    Y_init = np.empty(shape=(0, 0))
    
    baseline = LinearFeatureBaseline(env_spec=env_sim.spec)
    
    algo = TRPO(
            env=env_sim,
            policy=policy,
            baseline=baseline,
            batch_size=4000,
            max_path_length=1000,
            n_itr=10,
            discount=0.99,
            step_size=0.01,
    #         Uncomment both lines (this and the plot parameter below) to enable plotting
    #         plot=True,
        )
    for i in range(0,num_iters):
        print("iter: ", i)
#         algo.n_itr = algo.n_itr + test_iter
        current_mass = algo.env.wrapped_env.env.env.model.body_mass
        print("algo env mass: ", current_mass)
        algo.train()
        
        ################### Testing in GT parameter #################
#         print("algo sim mass: ", algo.env.wrapped_env.env.env.model.body_mass)
#         sampler_cls = BatchSampler
#         sampler = sampler_cls(algo)
#         sampler.start_worker()
#         paths = sampler.obtain_samples(1)
#         paths_returns = [sum(path["rewards"]) for path in paths]
#         sampler.shutdown_worker()
#         print("Sim sample ave returns:  \n\n\n\n", np.mean(paths_returns))   
        
        
#         print("algo sim mass: ", algo.env.wrapped_env.env.env.model.body_mass)
#         parallel_sampler.populate_task(algo.env, algo.policy, scope=algo.scope)
#         paths = parallel_sampler.sample_paths(
#             policy_params=algo.policy.get_param_values(),
#             max_samples=algo.batch_size,
#             max_path_length=algo.max_path_length,
#             env_params=algo.env.get_param_values(),
#             scope=algo.scope,
#         )
#         paths_returns = [sum(path["rewards"]) for path in paths]
#         parallel_sampler.terminate_task(scope=algo.scope)
#         print("Sim sample ave returns: ", np.mean(paths_returns), "\n\n\n\n")   
                
        env_real = normalize(GymEnv(env_name,record_video=False, record_log=False))
        mb = env_real.wrapped_env.env.env.model.body_mass
        mb = np.array(mb)
        mb = mb*0.6
        env_real.wrapped_env.env.env.model.body_mass = mb
        print("real mass: ", env_real.wrapped_env.env.env.model.body_mass)
        parallel_sampler.populate_task(env_real, algo.policy, scope=algo.scope)
        paths_real = parallel_sampler.sample_paths(
            policy_params=algo.policy.get_param_values(),
            max_samples=algo.batch_size,
            max_path_length=algo.max_path_length,
            env_params=env_real.get_param_values(),
            scope=algo.scope,
        )
        paths_real_returns = [sum(path["rewards"]) for path in paths_real]
        parallel_sampler.terminate_task(scope=algo.scope)
        ave_returns = np.mean(paths_real_returns)
        print("Real sample ave returns: ", ave_returns, "\n\n\n\n")
        returns.append(ave_returns)
        
#         algo_real = copy.deepcopy(algo)
#         algo_real.env = env_real
#         print("algo real mass: ", algo_real.env.wrapped_env.env.env.model.body_mass)
#         sampler_cls = BatchSampler
#         sampler = sampler_cls(algo_real)
#         sampler.start_worker()
#         paths = sampler.obtain_samples(1)
#         paths_returns = [sum(path["rewards"]) for path in paths]
#         sampler.shutdown_worker()
#         print("Real sample ave returns:", np.mean(paths_returns), "\n\n\n\n")   
#         returns.append(np.mean(paths_returns))
        
        env_real = normalize(GymEnv(env_name,record_video=False, record_log=False))
        mb = env_real.wrapped_env.env.env.model.body_mass
        mb = np.array(mb)
        mb = mb*0.6
        env_real.wrapped_env.env.env.model.body_mass = mb
        print("real mass: ", env_real.wrapped_env.env.env.model.body_mass)
        path = collect_data(env_real, algo.policy, max_path_length=1000,
                       animated=False, speedup=1)
        path["returns"] = sum(path["rewards"])
        print("real returns:  ", path['returns'], path['rewards'].shape , "\n\n\n\n")
#         returns.append(path['returns'])
        
        global states, observations, actions
        states = path['states']
        observations = path['observations']
        actions = path['actions']

        print("sim mass: ", algo.env.wrapped_env.env.env.model.body_mass)
        path_2 = rollout(algo.env, algo.policy, max_path_length=1000,
                       animated=False, speedup=1)
        path_2["returns"] = sum(path_2["rewards"])
        print("sim returns: ", path_2['returns'], path_2["rewards"].shape, "\n\n\n\n") 
        
        env_sim_test = normalize(GymEnv(env_name,record_video=False, record_log=False))
#         env_sim_test.wrapped_env.env.env.model.body_mass = np.array(current_mass)
        print("sim test mass: ", env_sim_test.wrapped_env.env.env.model.body_mass)
        path_2 = rollout(env_sim_test, algo.policy, max_path_length=1000,
                       animated=False, speedup=1)
        path_2["returns"] = sum(path_2["rewards"])
        print("sim test returns: ", path_2['returns'], path_2["rewards"].shape, "\n\n\n\n") 
        
#         if i==1:
#             model_body_mass = env_sim.wrapped_env.env.env.model.body_mass
#             bounds = []
#             for i in range(0,model_body_mass.shape[0]):
#                 bounds.append({'name': 'mass', 'type': 'continuous', 'domain': (model_body_mass[i,0]*0.9,model_body_mass[i,0]*1.1)})
        
        myBopt = BO(bounds, X_init, Y_init)
        X_init = myBopt.X
        Y_init = myBopt.Y
# #         algo.env.wrapped_env.env.env.model.body_mass = np.array(myBopt.x_opt)
        
        env_sim_new = normalize(GymEnv(env_name,record_video=False, record_log=False))        
#         ## update env_sim with new boday_mass from BO
        env_sim_new.wrapped_env.env.env.model.body_mass = np.array(myBopt.x_opt)
        algo = TRPO(
            env=env_sim_new,
            policy=policy,
            baseline=baseline,
            batch_size=4000,
            max_path_length=1000,
            n_itr=10,
            discount=0.99,
            step_size=0.01,
    #         Uncomment both lines (this and the plot parameter below) to enable plotting
    #         plot=True,
        )
        algo.env = env_sim_new        
        ##############################################################
        
    print("returns: ", returns)
    now = datetime.now()
    save_data_filename = "BO_returns_"+env_name+"_iter_"+str(total_iter)+"_"+str(now)
    with open(save_data_filename, 'wb') as f:
        pickle.dump(returns, f)
        
for v in variants:
    print("seed: ", v["seed"])
    run_experiment_lite(
        run_task,
        # Number of parallel workers for sampling
        n_parallel=1,
        # Only keep the snapshot parameters for the last iteration
        snapshot_mode="last",
        # Specifies the seed for the experiment. If this is not provided, a random seed
        # will be used
        seed=v["seed"],
        exp_prefix = "BO_trpo"+env_name+"_32_32_4000_10",
    #     plot=True,
    )


python /home/zhusj/repos/rllab/scripts/run_experiment_lite.py  --args_data 'gAJjY2xvdWRwaWNrbGUuY2xvdWRwaWNrbGUKX2ZpbGxfZnVuY3Rpb24KcQAoY2Nsb3VkcGlja2xlLmNsb3VkcGlja2xlCl9tYWtlX3NrZWxfZnVuYwpxAWNjbG91ZHBpY2tsZS5jbG91ZHBpY2tsZQpfYnVpbHRpbl90eXBlCnECWAgAAABDb2RlVHlwZXEDhXEEUnEFKEsASwBLGEsSS0djX2NvZGVjcwplbmNvZGUKcQZYBAYAAHQAAHQBAHQCAGQBAGQCAGQDAGQCAMKDAQLCgwEAfQEAdAMAZAQAfAEAagQAagUAagUAagYAagcAwoMCAAF8AQBqBABqBQBqBQBqBgBqBwB9AgBnAAB9AwB4XwB0CABkBQB8AgBqCQBkBQAZwoMCAERdRwB9BAB8AwBqCgBkBgBkBwBkCABkCQBkCgB8AgB8BABkBQBmAgAZZAsAFHwCAHwEAGQFAGYCABlkDAAUZgIAaQMAwoMBAAFxbABXdAsAZA0AfAEAagwAZA4AZDsAwoMAAn0FAHQNAGoOAGQQAGQ8AMKDAAF9BgB0DQBqDgBkEABkPQDCgwABfQcAdA8AZA0AfAEAagwAwoMAAX0IAHQQAGQRAHwBAGQSAHwFAGQTAHwIAGQUAGQVAGQWAGQXAGQYAGQZAGQaAGQbAGQcAGQdAMKDAAh9CQB4HgR0CABkBQB0EQDCgwIARF0NBH0EAHQDAGQeAHwEAMKDAgABfAkAagUAagQAagUAagUAagYAagcAfQoAdAMAZB8AfAoAwoMCAAF8CQBqEgDCgwAAAXQAAHQBAHQCAGQBAGQCAGQDAGQCAMKDAQLCgwEAfQsAfAsAagQAagUAagUAagYAagcAfQwAdA0AahMAfAwAwoMBAH0MAHwMAGQgABR9DAB8DAB8CwBqBABqBQBqBQBq

KeyboardInterrupt: 

In [31]:
env_name = "Swimmer-v1"
returns = []
total_iter = 400
test_iter = 10
num_iters = int(total_iter/test_iter)

run_experiment_lite(
    run_task,
    resume_from = "/home/zhusj/repos/rllab/data/local/trpo-Swimmer-v1-32-32-4000-10/trpo_Swimmer-v1_32_32_4000_10_2017_09_07_13_23_35_0006/params.pkl",
    # Number of parallel workers for sampling
#     n_parallel=16,
    # Only keep the snapshot parameters for the last iteration
#     snapshot_mode="last",
    # Specifies the seed for the experiment. If this is not provided, a random seed
    # will be used
#     seed=1,
    exp_prefix = "trpo_"+env_name+"_32_32_4000_10",
#     plot=True,
)

python /home/zhusj/repos/rllab/scripts/run_experiment_lite.py  --resume_from '/home/zhusj/repos/rllab/data/local/trpo-Swimmer-v1-32-32-4000-10/trpo_Swimmer-v1_32_32_4000_10_2017_09_07_13_23_35_0006/params.pkl'  --use_cloudpickle 'True'  --log_dir '/home/zhusj/repos/rllab/data/local/trpo-Swimmer-v1-32-32-4000-10/trpo_Swimmer-v1_32_32_4000_10_2017_09_07_13_23_35_0010'  --exp_name 'trpo_Swimmer-v1_32_32_4000_10_2017_09_07_13_23_35_0010'  --args_data 'gAJjY2xvdWRwaWNrbGUuY2xvdWRwaWNrbGUKX2ZpbGxfZnVuY3Rpb24KcQAoY2Nsb3VkcGlja2xlLmNsb3VkcGlja2xlCl9tYWtlX3NrZWxfZnVuYwpxAWNjbG91ZHBpY2tsZS5jbG91ZHBpY2tsZQpfYnVpbHRpbl90eXBlCnECWAgAAABDb2RlVHlwZXEDhXEEUnEFKEsASwBLEEsSS0djX2NvZGVjcwplbmNvZGUKcQZYUAMAAHQAAHQBAHQCAGQBAGQCAGQDAGQCAMKDAQLCgwEAYQMAdAQAZAQAdAMAagUAagYAagYAagcAaggAwoMCAAF0CQBqCgB0AwDCgwEAfQEAfAEAagUAagYAagYAagcAaggAfQIAdAsAagwAfAIAwoMBAH0CAHwCAGQFABR9AgB8AgB8AQBqBQBqBgBqBgBqBwBfCAB0BABkBgB8AQBqBQBqBgBqBgBqBwBqCADCgwIAAXQDAGoFAGoGAGoGAGoHAGoIAH0DAGcAAH0EAHhfAHQNAGQHAHwDAGoOAGQHABnCgwIARF1H

In [3]:
path = collect_data(env, policy, max_path_length=env.horizon,
                       animated=False, speedup=1)
path

{'actions': array([[ 0.15943319],
        [-1.63633099],
        [-0.94240068],
        [ 1.62921315]]), 'agent_infos': {'log_std': array([[ 0.03442661],
         [ 0.03442661],
         [ 0.03442661],
         [ 0.03442661]]), 'mean': array([[-0.16971114],
         [-0.17255606],
         [-0.18444359],
         [-0.35314067]])}, 'env_infos': {}, 'observations': array([[ -6.53789907e-03,  -4.04011712e-03,  -8.02106308e-03,
           7.73202508e-03],
        [ -3.33838927e-03,  -1.20304268e-02,   1.67452261e-01,
          -4.02749249e-01],
        [ -1.87152795e-02,   2.38020797e-02,  -9.32525536e-01,
           2.16314951e+00],
        [ -7.63754689e-02,   1.56155273e-01,  -1.94552484e+00,
           4.43404701e+00],
        [ -1.31829886e-01,   2.80571882e-01,  -8.39104008e-01,
           1.87326515e+00]]), 'rewards': array([ 1.,  1.,  1.,  1.])}

In [4]:
states = path['observations']

In [5]:
states

array([[ -6.53789907e-03,  -4.04011712e-03,  -8.02106308e-03,
          7.73202508e-03],
       [ -3.33838927e-03,  -1.20304268e-02,   1.67452261e-01,
         -4.02749249e-01],
       [ -1.87152795e-02,   2.38020797e-02,  -9.32525536e-01,
          2.16314951e+00],
       [ -7.63754689e-02,   1.56155273e-01,  -1.94552484e+00,
          4.43404701e+00],
       [ -1.31829886e-01,   2.80571882e-01,  -8.39104008e-01,
          1.87326515e+00]])

In [6]:
actions = path['actions']

In [7]:
actions

array([[ 0.15943319],
       [-1.63633099],
       [-0.94240068],
       [ 1.62921315]])

In [7]:
env2 = copy.deepcopy(env)

NameError: name 'env' is not defined

In [9]:
env2.wrapped_env.env.env.model.body_mass

array([[ 0.        ],
       [ 9.42477796],
       [ 4.8953899 ]])

In [10]:
mb = env2.wrapped_env.env.env.model.body_mass
mb = np.array(mb)
mb[2,0] = mb[1,0]*1.4
env2.wrapped_env.env.env.model.body_mass = mb

In [11]:
env2.reset()

array([ 0.00822978, -0.00634424, -0.00853528,  0.00116033])

In [12]:
env2.wrapped_env.env.env.set_state(states[0,:2],states[0,2:])

In [13]:
env2.wrapped_env.env.env.state_vector()

array([-0.0065379 , -0.00404012, -0.00802106,  0.00773203])

In [6]:
env2.reset()
env2.wrapped_env.env.env.set_state(states[0,:2],states[0,2:])
env2.wrapped_env.env.env.state_vector()
for i in range(0,actions.shape[0]):
    next_o, r, d, env_info = env2.step(actions[i,:])
    print (actions[i,:],next_o)

NameError: name 'env2' is not defined

In [16]:
env.wrapped_env.env.env.set_state(states[0,:2],states[0,2:])
for i in range(0,4):
    next_o, r, d, env_info = env.step(actions[i,:])
    print (actions[i,:],next_o)

[ 0.15943319] [-0.00333839 -0.01203043  0.16745226 -0.40274925]
[-1.63633099] [-0.01871528  0.02380208 -0.93252554  2.16314951]
[-0.94240068] [-0.07637547  0.15615527 -1.94552484  4.43404701]
[ 1.62921315] [-0.13182989  0.28057188 -0.83910401  1.87326515]
